In [1]:
## Initialise some variables and functions to be used
R.<x, y> = QQ[]
prec = 53
F2 = FiniteField(2)
from sage.groups.perm_gps.partn_ref.refinement_graphs import get_orbits
from collections import Counter
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

# Set the curves to calculate the orbits of characteristics on
fs = [y + y^3*x + x^3,
      1 + y^4 + x^4,
      x^4 + y^4 + x,
      1 + y^4 + x^4 + 7*(y^2 + x^2 + y^2*x^2),
      x^4 + y^4 + 1 + y^2,
      1 + y^3 + y*x^3,
      1 + y^4 + x^4 + 7*(y^2 + x^2) + 1*y^2*x^2,
      1 + y^2 + y^4 + x^3,
      1 + (y^3 + x^3) + y*x + 2*y^2*x^2,
      1 + y^4 + x^4 + (17*y^2 + 5*x^2) + 11*y^2*x^2,
      (1*y + 2*x) + (4*y^4 + 5*y^3*x + 6*y^2*x^2 + 7*y*x^3 + 8*y^4),
      1 + (1*y^2 + 2*y*x + 3*x^2) + (4*y^4 + 5*y^3*x + 6*y^2*x^2 + 7*y*x^3 + 8*y^4),
     ]
        
# Helper functions to be used in computing the orbits:
# n2v turns a number n into the corresponding binary vector
def n2v(n):
    return vector(Integer(n).digits(2, padto=2*g), F2)

# v2n turns a binary vector into a number.
def v2n(v):
    return ZZ(list(v), base=2)

# permutation_from_isomorphism takes a (binary) matrix corresponding to an element of the 
# rational representation mod 2 and computed the corresponding permutation of integers [0 .. 2^(2*g)-1]
def permutation_from_isomorphism(M):
    M = M.transpose()
    Jg = block_matrix([[matrix.zero(g), matrix.identity(g)], 
                       [-matrix.identity(g), matrix.zero(g)]])
    V = vector([sum([M[i,j]*M[i,k]*Jg[j,k] for j in range(2*g) for k in range(j, 2*g)]) 
                     for i in range(2*g)])
    return [v2n(M*n2v(n)+V) for n in range(2^(2*g))]

# orbits_to_counter takes the orbits under the action of a permutation group and returns 
# a counter of the number of orbits of a given size. 
def orbits_to_counter(perms):
    co = Counter()
    ce = Counter()
    for orbit in perms:
        if parity(n2v(orbit[0])):
            co[len(orbit)] += 1
        else:
            ce[len(orbit)] += 1
    print("{} =".format(2^(g-1)*(2^g-1)), [r"{}_{}".format(a, co[a]) 
                                           for a in sorted(list(co.keys()))])
    print("{} =".format(2^(g-1)*(2^g+1)), [r"{}_{}".format(a, ce[a]) 
                                           for a in sorted(list(ce.keys()))])
    return co, ce

# parity computes the parity of a binary vector. 
def parity(v):
    return sum([v[i]*v[i+g] for i in range(g)])
        
for f in fs:
    # Initialise the curve and compute the homomorphism basis
    print("f:", f)
    S = RiemannSurface(f, prec=prec)
    g = S.genus
    HB = S.homomorphism_basis(other=S)
    
    # Compute the representation of the automorphism group acting on homology
    Mlist = S.symplectic_isomorphisms(hom_basis=HB)
    # As the action on binary vectors depends only on the value of the rational rep mod 2, we only
    # require one of M and -M in the list of isomorphism of the Jacobian. 
    Mlist_trim = [Mlist[j] for j in set([min(i, Mlist.index(-Mlist[i])) for i in range(len(Mlist))])]
    
    if len(Mlist_trim)==1:
        print("Found automorphism group is trivial, investigate this case.")
        print("Observed orbits are thus trivial.")
        print()
        continue
    
    # Take a list of possible generators of the matrix group mod 2. 
    possible_gens = [M.change_ring(F2) for M in Mlist_trim if not (M.is_one() or (-M).is_one())]
    moments_dict = dict()

    # create a dictionary of the characteristic polynomials of the possible generators. 
    # This will provide a general purpose way of trimming the list of possible generators
    for A in possible_gens:
        moments_dict.update({tuple(A.charpoly().list()): A})

    candidate_gens = list(moments_dict.values())
    CG = MatrixGroup(candidate_gens)
    if CG.order() >= len(Mlist_trim):
        generators = candidate_gens
    else:
        generators = Mlist_trim
        print("warning: falling back to slow implementation with massive overcounting of generators")
 
    # Get the permutation group elements from the rational rep
    SS_Perms_exact = [permutation_from_isomorphism(M) 
                      for M in generators]

    # get the orbits
    SS_orbits_exact = get_orbits(SS_Perms_exact, 2^(2*g))
    print("Orbit decomposition:")
    co, ce = orbits_to_counter(SS_orbits_exact)
    ti = co[1]+ce[1]
    print("Total number of invariants:", ti)
    
    # Find the automorphism group of the curve as a permutation group subgroup of S_{2^{2g}}
    GP = PermutationGroup([[pi+1 for pi in perm] for perm in SS_Perms_exact])

    # Get the structure description
    SD = GP.structure_description()
    print("Structure description (reduced automorphism group):", SD)
    # The Gap structure description can fail to uniquely specify the group if it contains
    # a semi-direct product, so in this case we print the group id
    if ':' in SD:
        print("Group ID:", GP.group_id())
    
    oddi = ', '.join(["${}_{{{}}}$".format(a, co[a]) 
                       for a in sorted(list(co.keys()))])
    eveni = ', '.join(["${}_{{{}}}$".format(a, ce[a]) 
                       for a in sorted(list(ce.keys()))])
    
    print("Computing decomposition for (conjugacy classes) of subgroups")   
    # Find the subgroups up to conjugacy and calculate their orbit decomposition
    subgroups = GP.conjugacy_classes_subgroups()
    for H in subgroups[1:-1]:
        # Repeat the process of finding the orbits for each subgroup
        subgroup_orbits = get_orbits([[ii-1 for ii in gi.tuple()] 
                                      for gi in H.gens()], 2^(2*g))
        SD = H.structure_description()
        print("Structure description (subgroup):", SD)
        if ':' in SD:
            print("Group ID:", GP.group_id())
        print("Orbit decomposition:")
        _ = orbits_to_counter(subgroup_orbits)

    print()

f: x*y^3 + x^3 + y
Orbit decomposition:
28 = ['28_1']
36 = ['1_1', '7_2', '21_1']
Total number of invariants: 1
Structure description (reduced automorphism group): PSL(3,2)
Computing decomposition for (conjugacy classes) of subgroups
Structure description (subgroup): C2
Orbit decomposition:
28 = ['1_4', '2_12']
36 = ['1_12', '2_12']
Structure description (subgroup): C3
Orbit decomposition:
28 = ['1_1', '3_9']
36 = ['1_3', '3_11']
Structure description (subgroup): C2 x C2
Orbit decomposition:
28 = ['2_6', '4_4']
36 = ['1_8', '2_6', '4_4']
Structure description (subgroup): C2 x C2
Orbit decomposition:
28 = ['2_6', '4_4']
36 = ['1_8', '2_6', '4_4']
Structure description (subgroup): C4
Orbit decomposition:
28 = ['2_2', '4_6']
36 = ['1_4', '2_4', '4_6']
Structure description (subgroup): S3
Orbit decomposition:
28 = ['1_1', '3_3', '6_3']
36 = ['1_3', '3_9', '6_1']
Structure description (subgroup): C7
Orbit decomposition:
28 = ['7_4']
36 = ['1_1', '7_5']
Structure description (subgroup): D4
O

Orbit decomposition:
28 = ['1_1', '9_3']
36 = ['9_4']
Total number of invariants: 1
Structure description (reduced automorphism group): C9
Computing decomposition for (conjugacy classes) of subgroups
Structure description (subgroup): C3
Orbit decomposition:
28 = ['1_1', '3_9']
36 = ['3_12']

f: x^4 + x^2*y^2 + y^4 + 7*x^2 + 7*y^2 + 1
Orbit decomposition:
28 = ['4_5', '8_1']
36 = ['1_4', '2_4', '4_4', '8_1']
Total number of invariants: 4
Structure description (reduced automorphism group): D4
Computing decomposition for (conjugacy classes) of subgroups
Structure description (subgroup): C2
Orbit decomposition:
28 = ['1_4', '2_12']
36 = ['1_12', '2_12']
Structure description (subgroup): C2
Orbit decomposition:
28 = ['1_4', '2_12']
36 = ['1_12', '2_12']
Structure description (subgroup): C2
Orbit decomposition:
28 = ['1_4', '2_12']
36 = ['1_12', '2_12']
Structure description (subgroup): C2 x C2
Orbit decomposition:
28 = ['2_6', '4_4']
36 = ['1_8', '2_6', '4_4']
Structure description (subgrou